##### 版权所有 2018 TensorFlow 作者。

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 图像分类

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a></td>
</table>

本教程展示了如何使用`tf.keras.Sequential`模型对花朵图像进行分类，并使用`tf.keras.utils.image_dataset_from_directory`加载数据。它演示了以下概念：

- 有效地从磁盘加载数据集。
- 识别过拟合并应用技术来缓解它，包括数据增强和丢失。

本教程遵循基本的机器学习工作流程：

1. 检查和理解数据
2. 构建输入管道
3. 建立模型
4. 训练模型
5. 测试模型
6. 改进模型并重复该过程

此外，该笔记本还演示了如何将[保存的模型](../../../guide/saved_model.ipynb)转换为[TensorFlow Lite](https://www.tensorflow.org/lite/)模型，以便在移动、嵌入式和 IoT 设备上进行设备上机器学习。

## 设置

导入 TensorFlow 和其他必要的库：

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## 下载并探索数据集

本教程使用大约 3,700 张鲜花照片的数据集。数据集包含五个子目录，每个类一个：

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

下载后，您现在应该有可用的数据集副本。共有 3,670 张图像：

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

这里有一些玫瑰：

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
PIL.Image.open(str(roses[1]))

还有一些郁金香：

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
PIL.Image.open(str(tulips[1]))

## 使用 Keras 实用程序加载数据

接下来，使用有用的`tf.keras.utils.image_dataset_from_directory`实用程序从磁盘加载这些图像。只需几行代码，这将带您从磁盘上的图像目录到`tf.data.Dataset` 。如果您愿意，您还可以通过访问[加载和预处理图像](../load_data/images.ipynb)教程从头开始编写自己的数据加载代码。

### 创建数据集

为加载器定义一些参数：

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

在开发模型时使用验证拆分是一种很好的做法。将 80% 的图像用于训练，20% 用于验证。

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

您可以在这些数据集的`class_names`属性中找到类名。这些对应于按字母顺序排列的目录名称。

In [ ]:
class_names = train_ds.class_names
print(class_names)

## 可视化数据

以下是训练数据集中的前九幅图像：

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

您将在本教程后面将这些数据集传递给 Keras `Model.fit`方法进行训练。如果您愿意，您还可以手动迭代数据集并检索批量图像：

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

`image_batch`是形状`(32, 180, 180, 3)`的张量。这是一组 32 张形状为`180x180x3`的图像（最后一个维度是指颜色通道 RGB）。 `label_batch`是形状`(32,)`的张量，这些是 32 张图像的对应标签。

您可以在`image_batch`和`labels_batch`张量上调用`.numpy()`以将它们转换为`numpy.ndarray` 。


## 配置数据集以提高性能

确保使用缓冲预取，这样您就可以从磁盘产生数据而不会导致 I/O 阻塞。以下是加载数据时应使用的两种重要方法：

- `Dataset.cache`将图像在第一个时期从磁盘加载后保存在内存中。这将确保数据集在训练模型时不会成为瓶颈。如果您的数据集太大而无法放入内存，您还可以使用此方法创建高性能的磁盘缓存。
- `Dataset.prefetch`在训练时与数据预处理和模型执行重叠。

感兴趣的读者可以[在 tf.data API 指南的更好性能](../../guide/data_performance.ipynb)的*预取*部分了解更多关于这两种方法的信息，以及如何将数据缓存到磁盘。

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 标准化数据

RGB 通道值在`[0, 255]`范围内。这对于神经网络来说并不理想；一般来说，您应该设法使您的输入值变小。

在这里，您将使用`tf.keras.layers.Rescaling`将值标准化为`[0, 1]`范围内：

In [ ]:
normalization_layer = layers.Rescaling(1./255)

有两种方法可以使用该层。您可以通过调用`Dataset.map`将其应用于数据集：

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

或者，您可以在模型定义中包含该层，这可以简化部署。在这里使用第二种方法。

注意：您之前使用`tf.keras.utils.image_dataset_from_directory`的`image_size`参数调整了图像大小。如果您还想在模型中包含调整大小的逻辑，您可以使用`tf.keras.layers.Resizing`层。

## 一个基本的 Keras 模型

### 创建模型

Keras [Sequential](https://www.tensorflow.org/guide/keras/sequential_model)模型由三个卷积块 ( `tf.keras.layers.Conv2D` ) 组成，每个卷积块都有一个最大池化层 ( `tf.keras.layers.MaxPooling2D` )。有一个全连接层 ( `tf.keras.layers.Dense` )，上面有 128 个单元，由 ReLU 激活函数 ( `'relu'` ) 激活。该模型尚未针对高精度进行调整；本教程的目标是展示一种标准方法。

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

### 编译模型

对于本教程，选择`tf.keras.optimizers.Adam`优化器和`tf.keras.losses.SparseCategoricalCrossentropy`损失函数。要查看每个训练时期的训练和验证准确性，请将`metrics`参数传递给`Model.compile` 。

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### 型号汇总

使用`Model.summary`方法查看网络的所有层：

In [ ]:
model.summary()

### 训练模型

使用 Keras `Model.fit`方法训练模型 10 个 epoch：

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## 可视化训练结果

在训练集和验证集上创建损失和准确度图：

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

图表显示，训练准确度和验证准确度相差很大，模型在验证集上的准确度仅达到 60% 左右。

以下教程部分展示了如何检查出了什么问题并尝试提高模型的整体性能。

## 过拟合

在上图中，训练准确度随时间线性增加，而验证准确度在训练过程中停滞在 60% 左右。此外，训练和验证准确度之间的准确度差异很明显——这是[过度拟合](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit)的迹象。

当有少量训练示例时，模型有时会从训练示例中的噪声或不需要的细节中学习——在某种程度上它会对模型在新示例上的性能产生负面影响。这种现象被称为过拟合。这意味着该模型将很难在新数据集上进行泛化。

在训练过程中有多种方法可以对抗过度拟合。在本教程中，您将使用*数据增强*并将*dropout*添加到模型中。

## 数据增强

过度拟合通常发生在训练样本数量较少的情况下。[数据增强](./data_augmentation.ipynb)采用的方法是从现有示例中生成额外的训练数据，方法是使用随机变换来增强它们，从而产生看起来可信的图像。这有助于将模型暴露给数据的更多方面并更好地概括。

您将使用以下 Keras 预处理层实现数据增强： `tf.keras.layers.RandomFlip` 、 `tf.keras.layers.RandomRotation`和`tf.keras.layers.RandomZoom` 。这些可以像其他层一样包含在您的模型中，并在 GPU 上运行。

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

通过对同一图像多次应用数据增强来可视化一些增强示例：

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

在下一步训练之前，您将在模型中添加数据增强。

## 退出

另一种减少过拟合的技术是向网络引入[dropout](https://developers.google.com/machine-learning/glossary#dropout_regularization) {:.external} 正则化。

当您将 dropout 应用于层时，它会在训练过程中从层中随机丢弃（通过将激活设置为零）一些输出单元。 Dropout 将一个小数作为其输入值，例如 0.1、0.2、0.4 等。这意味着从应用层中随机丢弃 10%、20% 或 40% 的输出单元。

在使用增强图像对其进行训练之前，使用`tf.keras.layers.Dropout`创建一个新的神经网络：

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

## 编译和训练模型

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## 可视化训练结果

应用数据增强和`tf.keras.layers.Dropout`后，过拟合比以前少，训练和验证准确率更接近：

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 预测新数据

使用您的模型对未包含在训练或验证集中的图像进行分类。

注意：数据增强和丢失层在推理时是不活动的。

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

## 使用 TensorFlow Lite

TensorFlow Lite 是一组工具，可帮助开发人员在移动、嵌入式和边缘设备上运行模型，从而实现设备上机器学习。

### 将 Keras Sequential 模型转换为 TensorFlow Lite 模型

要将经过训练的模型与设备上的应用程序一起使用，首先[将其转换](https://www.tensorflow.org/lite/models/convert)为更小、更高效的模型格式，称为[TensorFlow Lite](https://www.tensorflow.org/lite/)模型。

在此示例中，采用经过训练的 Keras Sequential 模型并使用`tf.lite.TFLiteConverter.from_keras_model`生成[TensorFlow Lite](https://www.tensorflow.org/lite/)模型：

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

您在上一步中保存的 TensorFlow Lite 模型可以包含多个函数签名。 Keras 模型转换器 API 自动使用默认签名。详细了解[TensorFlow Lite 签名](https://www.tensorflow.org/lite/guide/signatures)。

### 运行 TensorFlow Lite 模型

您可以通过`tf.lite.Interpreter`类在 Python 中访问 TensorFlow Lite 保存的模型签名。

使用`Interpreter`加载模型：

In [ ]:
TF_MODEL_FILE_PATH = 'model.tflite' # The default path to the saved TensorFlow Lite model

interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)

从转换后的模型中打印签名以获得输入（和输出）的名称：


In [ ]:
interpreter.get_signature_list()

在此示例中，您有一个名为`serving_default`的默认签名。此外， `'inputs'`的名称是`'sequential_1_input'` ，而`'outputs'`则称为`'outputs'` 。您可以在运行`Model.summary`时查找这些第一个和最后一个 Keras 层名称，如本教程前面所述。

现在，您可以使用`tf.lite.Interpreter.get_signature_runner`通过传递签名名称对示例图像执行推理来测试加载的 TensorFlow 模型，如下所示：

In [ ]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

与您在本教程前面所做的类似，您可以使用 TensorFlow Lite 模型对未包含在训练或验证集中的图像进行分类。

您已经对该图像进行了张量化并将其保存为`img_array` 。现在，将其传递给加载的 TensorFlow Lite 模型 ( `predictions_lite` ) 的第一个参数（ `'inputs'`的名称），计算 softmax 激活，然后打印具有最高计算概率的类的预测。

In [ ]:
predictions_lite = classify_lite(sequential_1_input=img_array)['outputs']
score_lite = tf.nn.softmax(predictions_lite)

assert np.allclose(predictions, predictions_lite)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score_lite)], 100 * np.max(score_lite))
)

在`'daisy'` 、 `'dandelion'` 、 `'roses'` 、 `'sunflowers'`和`'tulips'`这五个类别中，模型应该预测图像属于向日葵，这与 TensorFlow Lite 转换之前的结果相同。


## 下一步

本教程展示了如何训练用于图像分类的模型、对其进行测试、将其转换为 TensorFlow Lite 格式以用于设备上的应用程序（例如图像分类应用程序），以及使用 Python API 使用 TensorFlow Lite 模型执行推理。

您可以通过[教程](https://www.tensorflow.org/lite/tutorials)和[指南](https://www.tensorflow.org/lite/guide)了解有关 TensorFlow Lite 的更多信息。